In [16]:
import os
from pathlib import Path

import gspread
import pandas as pd
from dotenv import load_dotenv
from elevenlabs import play
from elevenlabs.client import ElevenLabs
from google.oauth2.service_account import Credentials

load_dotenv()

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

In [4]:
root_dir = Path().resolve().parent
cache_dir = root_dir / ".cache" / "tts_test"

In [5]:
sa_filepath = root_dir / "sa-key.json"

# 認証情報の設定
scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

# サービスアカウントキーのパスを指定
creds = Credentials.from_service_account_file(sa_filepath, scopes=scopes)

# gspreadクライアントの初期化
client = gspread.authorize(creds)

In [8]:
sheet_key = "1bzROAf4BHIdq551QpWbRL1xRnB2hFqJGQ9Mu9Vj0w-c"

In [ ]:
# スプレッドシートを開く(URLまたはキーで指定)
spreadsheet = client.open_by_key(sheet_key)

# 特定のワークシートを取得
worksheet = spreadsheet.worksheet("scenes")

# 全データを取得してDataFrameに変換
df = pd.DataFrame(worksheet.get_all_records())

In [11]:
df["target_sec"] = df["end_sec"] - df["start_sec"]

In [12]:
df

,title,content,telop,image_search_query,start_sec,end_sec,process_index,target_sec
0,Scene 1 導入：海と夜景が日常の街,海と観覧車の夜景、レトロな街並み、ディープな飲み屋街までそろうエリアを、住む目線でまとめて紹...,海と夜景が日常の街,横浜 みなとみらい 夜景 街並み,0.0,1.5,0,1.5
1,Scene 2 駅名紹介：横浜駅,今回紹介するのは、JR東海道本線や湘南新宿ラインなどが使える『横浜駅』エリアです。,今日の住みたい駅は横浜駅,横浜駅 東口 西口 駅前 俯瞰,1.5,3.0,1,1.5
2,Scene 3 全体像：海も繁華街も近い街,横浜駅を起点に、海辺の高層ビル街からレトロな飲み屋街までが徒歩や1〜2駅圏内に集まり、エリア...,海も繁華街も揃う横浜生活,横浜駅 周辺地図 みなとみらい 野毛 中華街 俯瞰,3.0,4.5,2,1.5
3,Scene 4 アクセス：都心への通勤・通学,横浜駅からは東京駅へ約25分、新宿駅へ約31分、品川駅へ約12分と、主要ターミナルに乗換なし...,東京25分・新宿31分圏内,JR東海道線 湘南新宿ライン 車内 路線図 横浜から東京 新宿,4.5,6.5,3,2.0
4,Scene 5 海辺カフェ＆夜景エリア,みなとみらいや東口ポートサイド周辺には、海を望むテラスカフェやスイーツ、ホテルラウンジが集ま...,海辺カフェと夜景スポット充実,横浜 みなとみらい 海辺カフェ テラス 夜景 ランドマークタワー,6.5,8.5,4,2.0
5,Scene 6 元町・中華街〜山下公園,少し足を伸ばすと、横浜中華街や元町エリアで中華料理や食べ歩きスイーツが楽しめ、山下公園や港の...,中華街グルメと海辺散歩圏内,横浜中華街 山下公園 食べ歩き 海 桟橋,8.5,10.5,5,2.0
6,Scene 7 野毛・西口の飲み歩き文化,桜木町〜野毛エリアや横浜駅西口一番街には、立ち飲みや大衆酒場、町中華、ネオ酒場が路地にびっし...,レトロ酒場からネオ酒場まで,横浜 野毛 飲み屋街 提灯 路地 夜景,10.5,12.5,6,2.0
7,Scene 8 まとめ：暮らしのバランス,横浜駅周辺は、都心へのアクセスの良さに加え、海辺カフェ、異国情緒ある食べ歩きスポット、ディー...,アクセス良好で遊びも充実,横浜駅 周辺 街歩き みなとみらい 野毛 中華街,12.5,14.0,7,1.5
8,Scene 9 締め：ここで暮らすイメージ,海風を感じる日中と、ネオンが灯る夜の表情を楽しめる横浜駅エリアで、自分らしい暮らし方をイメー...,海風感じる横浜暮らし,横浜 夜景 高層マンション 海沿い 生活イメージ,14.0,15.0,8,1.0


In [17]:
client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

# 日本語音声生成に最適化されたパラメータ
audio = client.text_to_speech.convert(
    text="海と観覧車の夜景、レトロな街並み、ディープな飲み屋街までそろうエリアを、住む目線でまとめて紹介します。",
    voice_id="fUjY9K2nAIwlALOwSiwc",  # 日本語対応のvoice_idを使用
    model_id="eleven_v3",  # 最新モデル
    output_format="mp3_44100_128",  # 標準的な音質
    language_code="ja",  # 日本語を明示的に指定
)

In [ ]:
# 音声をファイルに保存
output_path = cache_dir / "test_audio.mp3"
cache_dir.mkdir(parents=True, exist_ok=True)

with open(output_path, "wb") as f:
    for chunk in audio:
        f.write(chunk)

print(f"音声を保存しました: {output_path}")

# 保存した音声を再生


In [19]:
play(audio)

TypeError: 'module' object is not callable

In [ ]:
client = ElevenLabs(api_key="YOUR_API_KEY")

audio = client.text_to_speech.convert(
    text="The first move is what sets everything in motion.",
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

play(audio)

TypeError: 'module' object is not callable